# 생태통로 주변 산책로까지의 최단 거리

## 패키지

In [3]:
!pip install pymysql
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from haversine import haversine

## 데이터 로드

In [5]:
endpoint = 'dao.cdqd2bwz9q4s.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [6]:
def sql_to_df(table_name):
    sql = "SELECT * FROM {}".format(table_name)
    df = pd.read_sql(sql, db_connection)
    return df

In [7]:
main = sql_to_df("MainData")
trkroad = sql_to_df("산책로_2013")

In [8]:
main.head()

,번호,위도_십진법,경도_십진법,생태통로타입,차선_수,등산객이용빈도,생태통로효율성,"이용확인종,이용빈도높은종,멸종위기종이용빈도",이용확인 종 수,면적_m^2,너비_m,길이_m
0,1495,35.36019,128.06375,0,1,1,3,None,0,NaN,NaN,NaN
1,1277,37.03408,128.40661,1,0,0,3,None,0,NaN,NaN,NaN
2,1278,36.89832,128.30563,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN
3,1279,36.89687,128.30581,1,0,0,1,고라니 멧돼지 파충류,3,NaN,NaN,NaN
4,1090,36.71336,127.42200,0,2,5,1,개 고양이 다람쥐,3,312.74,8.23,38.0


In [9]:
main.shape

(497, 12)

In [12]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   번호                       497 non-null    int64  
 1   위도_십진법                   497 non-null    float64
 2   경도_십진법                   497 non-null    float64
 3   생태통로타입                   497 non-null    int64  
 4   차선_수                     497 non-null    int64  
 5   등산객이용빈도                  497 non-null    int64  
 6   생태통로효율성                  497 non-null    int64  
 7   이용확인종,이용빈도높은종,멸종위기종이용빈도  211 non-null    object 
 8   이용확인 종 수                 497 non-null    int64  
 9   면적_m^2                   240 non-null    float64
 10  너비_m                     240 non-null    float64
 11  길이_m                     240 non-null    float64
dtypes: float64(5), int64(6), object(1)
memory usage: 46.7+ KB


In [11]:
trkroad

,좌표,산책로구분,명칭,코스번호,코스명,코스난이도,소요시간,코스길이(km)
0,"[[127.7846719856947, 35.16237732803098], [127....",둘레길링크,하동호-삼화실,11코스,지리산둘레길,하,←4시간 9.3Km→,9342.42811403
1,"[[127.65931897718815, 35.18070376552526], [127...",둘레길링크,원부춘-가탄,15코스,지리산둘레길,상,←7시간30분 12.6Km→,13006.1001180
2,"[[127.75974281501351, 35.10290440337887], [127...",둘레길링크,하동읍-서당,13코스,지리산둘레길,하,←2시간30분 7.1Km→,7015.43251329
3,"[[127.58708082167367, 35.21776381053337], [127...",둘레길링크,목아재-당재,17코스,지리산둘레길,중,←3시간 7.8Km→,7909.00839247
4,"[[127.60337954723201, 35.42996623453473], [127...",둘레길링크,인월-금계,3코스,지리산둘레길,중,←8시간 19.3Km→,20457.8570286
5,"[[127.53082624461805, 35.440650005656984], [12...",둘레길링크,운봉-인월,2코스,지리산둘레길,중,←4시간 9.4Km→,9372.14217937
6,"[[127.4454921074571, 35.39082602777359], [127....",둘레길링크,주천-운봉,1코스,지리산둘레길,중,←6시간 14.3Km→,14564.6135652
7,"[[127.4635552642758, 35.26121956128309], [127....",둘레길링크,방광-산동,21코스,지리산둘레길,상,←6시간 13Km→,12770.7762270
8,"[[127.51500422178758, 35.205644472010604], [12...",둘레길링크,송정-오미,18코스,지리산둘레길,중,←5시간30분 9.2Km→,9875.17087729
9,"[[127.5728033822975, 35.20605879726299], [127....",둘레길링크,가탄-송정,16코스,지리산둘레길,상,←6시간30분 11.3Km→,9964.21062247


In [14]:
# 리스트 형태 문자열을 리스트로 변환
for i in range(len(trkroad)):
  trkroad.loc[i, '좌표'] = eval(trkroad.loc[i, '좌표'])

In [15]:
trkroad.shape

(10, 8)

In [16]:
trkroad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   좌표        10 non-null     object
 1   산책로구분     10 non-null     object
 2   명칭        10 non-null     object
 3   코스번호      10 non-null     object
 4   코스명       10 non-null     object
 5   코스난이도     10 non-null     object
 6   소요시간      10 non-null     object
 7   코스길이(km)  10 non-null     object
dtypes: object(8)
memory usage: 768.0+ bytes


## 거리 계산

In [17]:
# A-B 지점 간 거리(km) 계산 함수
def distance(A_lat, A_long, B_lat, B_long): # A 지점 위도 및 경도, B 지점 위도 및 경도
    A = (A_lat, A_long)
    B = (B_lat, B_long)
    dist = haversine(A, B, unit='km')
    return dist

In [18]:
# test
distance(main.loc[0, '위도_십진법'], main.loc[0, '경도_십진법'], trkroad.loc[0, '좌표'][0][1], trkroad.loc[0, '좌표'][0][0])

33.553784931088884

In [19]:
dist_list = []
i_list = []
j_list = []
for i, m_row in main.iterrows(): # 각 생태통로 반복
  for j, tr_row in trkroad.iterrows(): # 각 산책로 반복
    for tr_coord in tr_row['좌표']: # 각 산책로의 좌표 리스트 
      dist_list.append(distance(m_row['위도_십진법'], m_row['경도_십진법'], tr_coord[1], tr_coord[0]))
      i_list.append(i)
      j_list.append(j)

In [20]:
dist_df = pd.DataFrame({'main_idx': i_list,
                        'trkroad_idx': j_list,
                        'distance': dist_list})
dist_df

,main_idx,trkroad_idx,distance
0,0,0,33.553785
1,0,0,33.441964
2,0,0,33.310766
3,0,0,33.336303
4,0,0,33.410768
...,...,...,...
394613,496,9,161.883422
394614,496,9,161.862358
394615,496,9,161.822448
394616,496,9,161.808255


## 생태통로별 산책로까지의 최단 거리

In [26]:
shortest = pd.DataFrame({'산책로까지의 최단 거리(km)': dist_df.groupby(['main_idx'])['distance'].min()}).reset_index(drop=True)
shortest

,산책로까지의 최단 거리(km)
0,32.993820
1,189.603501
2,172.236247
3,172.092457
4,140.219401
...,...
492,140.166888
493,140.681388
494,140.066912
495,145.890079


## 최종 업로드 할 데이터프레임

In [27]:
mdyr = sql_to_df("MainData_inner_예림")
mdyr

,번호,주변 로드킬 빈도
0,1495,0
1,1277,0
2,1278,0
3,1279,0
4,1090,5
...,...,...
492,1089,5
493,1087,5
494,1088,5
495,1091,0


In [28]:
final = pd.concat([mdyr, shortest], axis=1)
final

,번호,주변 로드킬 빈도,산책로까지의 최단 거리(km)
0,1495,0,32.993820
1,1277,0,189.603501
2,1278,0,172.236247
3,1279,0,172.092457
4,1090,5,140.219401
...,...,...,...
492,1089,5,140.166888
493,1087,5,140.681388
494,1088,5,140.066912
495,1091,0,145.890079


In [29]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   번호                497 non-null    int64  
 1   주변 로드킬 빈도         497 non-null    int64  
 2   산책로까지의 최단 거리(km)  497 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 11.8 KB


## RDS에 데이터프레임 업로드

In [30]:
final.to_sql(name='MainData_inner_예림', con=db_connection, if_exists='replace', index=False) # if_exists options : append, fail, replace